### 크롤링 준비

#### 1. 작업 폴더 설정

```R
install.packages("rstudioapi")   # rstudioapi 설치                         
setwd(dirname(rstudioapi::getSourceEditorContext()$path)) # 작업폴더 설정
## setwd("C:/Users/이찬솔/Documents/Shiny/Data")
getwd()   # 확인
```

#### 2. 수집 대상 지역 설정

- 지역코드 : 행안부의 행정표준코드관리시스템(www.code.go.kr) 기준

In [6]:
setwd(paste0(getwd(), "/Data"))

ERROR: Error in setwd(paste0(getwd(), "/Data")): cannot change working directory


In [20]:
library(tidyverse)
loc <- read.csv("./01_code/sigun_code/sigun_code.csv", fileEncoding="UTF-8")  #  지역코드
loc$code <- as.character(loc$code) # 행정구역명 문자 변환
loc %>% head

,code,sido,sigungu,addr_1,addr_2
,<chr>,<chr>,<chr>,<chr>,<chr>
1,11110,서울특별시,종로구,서울_종로,서울특별시 종로구
2,11140,서울특별시,중구,서울_중구,서울특별시 중구
3,11170,서울특별시,용산구,서울_용산,서울특별시 용산구
4,11200,서울특별시,성동구,서울_성동,서울특별시 성동구
5,11215,서울특별시,광진구,서울_광진,서울특별시 광진구
6,11230,서울특별시,동대문구,서울_동대문,서울특별시 동대문구


#### 3. 수집 기간 설정

In [9]:
datelist <- seq(from = as.Date("2021-01-01"),
                 to = as.Date("2021-12-31"),
                 by = '1 month')
datelist <- format(datelist, format = '%Y%m')
datelist[1:3]

[1] "202101" "202102" "202103"

#### 4. 인증키 입력

In [10]:
service_key <- "%2Bcn9rw%2FlGxu%2F0wJDFFD4weYYvWPMiQuoBlillCf5H%2Fq64%2B7d4wexYt7ik7uIhWiKBHNfr5cDEGNwl41mOvoraw%3D%3D"

### 요청 목록 생성
#### 1. 요청 목록 생성

In [11]:
url_list <- list() # 빈 리스트 만들기
cnt <- 0           # 반복문의 제어 변수 초깃값 설정

#### 2. 요청 목록 채우기

In [12]:
datelist %>% head

[1] "202101" "202102" "202103" "202104" "202105" "202106"

In [ ]:
for(i in 1:nrow(loc)){           # 외부반복: 25개 자치구
  for(j in 1:length(datelist)){  # 내부반복: 12개월
    cnt <- cnt + 1               # 반복누적 카운팅
    # 요청 목록 채우기 (25 X 12= 300)
    url_list[cnt] <- paste0("http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?",
                            "LAWD_CD=", loc[i,1],         # 지역코드
                            "&DEAL_YMD=", datelist[j],    # 수집월
                            "&numOfRows=", 100,           # 한번에 가져올 최대 자료 수
                            "&serviceKey=", service_key)  # 인증키
  } 
  Sys.sleep(0.1)   # 0.1초간 멈춤
  msg <- paste0("[", i,"/",nrow(loc), "]  ", loc[i,3], " 의 크롤링 목록이 생성됨 => 총 [", cnt,"] 건") # 알림 메시지
  cat(msg, "\n\n") 
}

#### 요청 목록 확인

In [14]:
url_list[1]

[[1]]
[1] "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?LAWD_CD=11110&DEAL_YMD=202101&numOfRows=100&serviceKey=%2Bcn9rw%2FlGxu%2F0wJDFFD4weYYvWPMiQuoBlillCf5H%2Fq64%2B7d4wexYt7ik7uIhWiKBHNfr5cDEGNwl41mOvoraw%3D%3D"

In [15]:
length(url_list)                # 요청목록 갯수 확인
browseURL(paste0(url_list[1]))  # 정상작동 확인(웹브라우저 실행)

[1] 300

- 브라우저의 ```<resultMsg>``` 태그에 "NORMAL SERVICE"라고 나오면 정상 동작 
- 만약 다른 메시지가 나오고 아래쪽에 데이터가 비어 있다면 공공데이터포털에서 제공하는 기술문서에서 'OPEN API 에러 코드별 조치 방안'참조

### 크롤러 제작
#### 1. 임시 저장 리스트 생성

In [17]:
library(XML)        
library(data.table) 
library(stringr)    

### ```dir.create("폴더")``` : 폴더 생성

In [18]:
raw_data <- list()        # xml 임시 저장소
root_Node <- list()       # 거래내역 추출 임시 저장소
total <- list()           # 거래내역 정리 임시 저장소
dir.create("02_raw_data") 

#### 2. 자료 요청하고 응답 받기

In [ ]:
#---# [2단계: URL 요청 - XML 응답]

for(i in 1:length(url_list)){   # 요청목록(url_list) 반복
  raw_data[[i]] <- xmlTreeParse(url_list[i], useInternalNodes = TRUE, encoding = "UTF-8") # 결과 저장
  root_Node[[i]] <- xmlRoot(raw_data[[i]]) # xmlRoot로 추출
  
#---# [3단계: 전체 거래 건수 확인]
  
  items <- root_Node[[i]][[2]][['items']]  # 전체 거래내역(items) 추출
  size <- xmlSize(items)                   # 전체 거래 건수 확인    

#---# [4단계: 거래 내역 추출]
  
  item <- list()  # 전체 거래내역(items) 저장 임시 리스트 생성
  item_temp_dt <- data.table()  # 세부 거래내역(item) 저장 임시 테이블 생성
  Sys.sleep(.1)  # 0.1초 멈춤
    
  for(m in 1:size){  # 전체 거래건수(size)만큼 반복
      
    #---# 세부 거래내역 분리   
    item_temp <- xmlSApply(items[[m]],xmlValue)
    item_temp_dt <- data.table(year = item_temp[4],     # 거래 년 
                               month = item_temp[7],    # 거래 월
                               day = item_temp[8],      # 거래 일
                               price = item_temp[1],    # 거래금액
                               code = item_temp[12],    # 지역코드
                               dong_nm = item_temp[5],  # 법정동
                               jibun = item_temp[11],   # 지번
                               con_year = item_temp[3], # 건축연도 
                               apt_nm = item_temp[6],   # 아파트 이름   
                               area = item_temp[9],     # 전용면적
                               floor = item_temp[13])   # 층수 
    item[[m]] <- item_temp_dt}    # 분리된 거래내역 순서대로 저장
  apt_bind <- rbindlist(item)     # 통합 저장

#---# [5단계: 응답 내역 저장]
  
  region_nm <- subset(loc, code == str_sub(url_list[i],115, 119))$addr_1 # 지역명 추출
  month <- str_sub(url_list[i],130, 135)   # 연월(YYYYMM) 추출
  path <- as.character(paste0("./02_raw_data/", region_nm, "_", month,".csv")) # 저장위치 설정
  write.csv(apt_bind, path)     # csv 저장
  msg <- paste0("[", i,"/",length(url_list), "] 수집한 데이터를 [", path,"]에 저장 합니다.") # 알림 메시지
  cat(msg, "\n\n")
}   

# 

## ```XML```

In [21]:
URL <- "https://d396qusza40orc.cloudfront.net/getdata%2Fdata%2Frestaurants.xml"

### ```XML::xmlTreeParse()``` : URL을 XML로 저장
### ```XML::xmlRoot()``` : XML의 루트 노드만 추출
### ```XML::xmlSize()``` : XML의 크기

In [31]:
file <- xmlTreeParse(sub("s", "", URL), useInternal = T)

### ```XML::xmlToDataFrame()``` : 저장된 XML을 데이터프레임으로 변환

In [23]:
(file <- xmlToDataFrame(file))

row,NA,NA.1,NA.2,NA.3,NA.4,NA.5,NA.6,NA.7,NA.8,...,NA,NA.1,NA.2,NA.3,NA.4,NA.5,NA.6,NA.7,NA.8,NA.9
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,...,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
41021206Frankford2NORTHEASTERN,191921231Fells Point1SOUTHEASTERN,SAUTE21224Canton1SOUTHEASTERN,#1 CHINESE KITCHEN21211Hampden14NORTHERN,#1 chinese restaurant21223Millhill9SOUTHWESTERN,19TH HOLE21218Clifton Park14NORTHEASTERN,3 KINGS21205McElderry Park13SOUTHEASTERN,"3 MILES HOUSE, INC.21211Remington7NORTHERN",3 W'S TAVERN21205McElderry Park13SOUTHEASTERN,300 SOUTH ANN STREET21231Upper Fells Point1SOUTHEASTERN,...,YELLOW DOG TAVERN21224Canton1SOUTHEASTERN,YETI RESTAURANT & CARRYOUT21212Rosebank4NORTHERN,YORK CLUB TAVERN21212Homeland4NORTHERN,YOUNG'S PLACE21231Middle East13SOUTHEASTERN,ZEEBA LOUNGE21230Federal Hill10SOUTHERN,ZEN WEST ROADSIDE CANTINA21212Rosebank4NORTHERN,ZIASCOS21231Washington Hill1SOUTHEASTERN,ZINK'S CAF혨21213Belair-Edison13NORTHEASTERN,ZISSIMOS BAR21211Hampden7NORTHERN,ZORBAS21224Greektown2SOUTHEASTERN


In [24]:
file <- as.data.frame(t(file))
file %>% head

,V1
,<chr>
row,41021206Frankford2NORTHEASTERN
NA.,191921231Fells Point1SOUTHEASTERN
NA..1,SAUTE21224Canton1SOUTHEASTERN
NA..2,#1 CHINESE KITCHEN21211Hampden14NORTHERN
NA..3,#1 chinese restaurant21223Millhill9SOUTHWESTERN
NA..4,19TH HOLE21218Clifton Park14NORTHEASTERN


### ```xmlSApply(rootNode,xmlValue)``` : XML 모든 엘리먼트에 대해 루프를 돌면서 해당 엘리멘트가 가지고 있는 값들을 추출한다.

### 복수의 리스트를 데이터 프레임으로 변환
#### ```data.table::rbindlist()```
#### ```plyr::ldply()```

## 
### 자료정리 : 자료 통합

- 02_raw_data 폴더에 있는 파일 목록

In [28]:
files <- dir("./02_raw_data")    
files %>% head

[1] "서울_강남_202101.csv" "서울_강남_202102.csv" "서울_강남_202103.csv"
[4] "서울_강남_202104.csv" "서울_강남_202105.csv" "서울_강남_202106.csv"

- 통합된 자료를 저장할 03_integrated 폴더 생성

In [37]:
apt_price <- plyr::ldply(as.list(paste0("./02_raw_data/", files)), read.csv) # 모든 파일 하나로 결합
apt_price %>% head

,X,year,month,day,price,code,dong_nm,jibun,con_year,apt_nm,area,floor
,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<dbl>,<chr>
1,1,2021,1,5,"31,000",11680,역삼동,720-25,2002,대우디오빌,30.03,4
2,2,2021,1,6,"61,000",11680,역삼동,766-8,2002,트레벨,33.48,3
3,3,2021,1,7,"198,000",11680,역삼동,757,2005,역삼래미안,59.73,14
4,4,2021,1,7,"193,800",11680,역삼동,757,2005,역삼래미안,59.40,16
5,5,2021,1,9,"91,000",11680,역삼동,796-29,2012,강남서해더블루,66.04,10
6,6,2021,1,11,"80,000",11680,역삼동,783-3,2002,갤러리하우스,84.95,1


In [ ]:
dir.create("./03_integrated")
save(apt_price, file = "./03_integrated/03_apt_price.rdata") 
write.csv(apt_price, "./03_integrated/03_apt_price.csv")   